## 참조자료 

https://stevedonovan.github.io/rustifications/2018/08/18/rust-closures-are-hard.html

# 1. 클로저 표현식

- 클로저 표현식은  고유한 익명 타입의 클로저 값을 생성합니다. 
- 클로저 유형은 캡처된 변수를 포함하는 구조체와 거의 동일합니다.

## Rust 클로저 특성
Rust 프로그래밍 언어에서 Fn, FnMut, FnOnce는 클로저와 다른 유형의 함수 객체에 대한 다양한 동작을 정의하는 특성입니다.

- Fn 특성은 여러 번 호출할 수 있고 불변으로 차용할 수 있는 클로저를 나타냅니다. 이 특성에는 self에 대한 참조를 차용하는 하나의 연관된 메서드 호출(&self, args)이 있으며, 이러한 요구 사항을 충족하는 모든 클로저에서 구현할 수 있습니다.

- FnMut 형질은 여러 번 호출할 수 있고 가변적으로 차용할 수 있는 클로저를 나타냅니다. 이 형질에는 self에 대한 가변 참조를 취하는 call_mut(&mut self, args) 메서드가 하나 있으며, 이러한 요구 사항을 충족하는 모든 클로저에 의해 구현될 수 있습니다.

- FnOnce 형질은 한 번만 호출할 수 있는 클로저를 나타냅니다. 이 특성에는 self의 소유권을 취하는 call_once(self, args) 메서드가 하나 있으며, 이러한 요구 사항을 충족하는 모든 클로저에서 구현할 수 있습니다.


## 1-1 클로저 정의 및 호출하기 

- 주의할 점은 구조체 등은 소유권이 이동하므로 클로저 내부 사용하면 예외 발생 

### 클로저 정의 

-  매개변수는 |매개변수 | 정의하고 
-  반환자료형은  -> 다음에 정의
-  본문은 { } 사이에 정의한다. 

In [5]:
fn main() {
    let m = 1.0;
    let c = 2.0;
    let _z = String::from("가가가"); 

    let line = |x:f64|->f64 {      // 변수에 클로저 할당할 때는 main 함수 내부에 정의 할 것
        let s = m*x + c;           // 함수의 지역변수를 클로저에서 사용이 가능 : 정수는 이동되지 않아서 사용가능 
        //let zz = _z;             // _z는 한번 소유권 이동이 되어서 계속 호출시 문제됨 ...
        s                          // 내부에서 소유권 변동을 주의해야 함  
    };
    println!("{} {}", line(0.0), line(1.0));
    println!("m {:.2} ", m);
    // println!(" z { } ", z);
}

In [6]:
main()

2 3
 m 1.00 


()

## 1-2 클로저 타입 알아보기

- https://doc.rust-lang.org/reference/types/closure.html

### 변수에 클로저의 타입을 정의할 때는
- 함수포인터 타입인 fn을 사용해서 정의한다
- 이때는 캡처되는 외부 변수가 없어야 한다

In [6]:
{
    let add_one_v2: fn(u32)->u32 = |x: u32| -> u32 { x + 1 };
    
    println!(" {} ", add_one_v2(100));
    
};

 101 


### 클로저에서 외부 변수를 캡처할 때는 함수포인터타입으로 처리할 수 없다. 

In [7]:
{
    let cap_var = 300; 
    let add_one_v2: fn(u32)->u32 = |x: u32| -> u32 { x + cap_var};
    
    println!(" {} ", add_one_v2(100));
    
};

Error: mismatched types



## 클로저가 환경에서 값을 캡처하는 세 가지 방식Permalink
소유권 받기, 불변으로 빌려오기, 가변으로 빌려오는 방식이다.

FnOnce는 캡처한 변수의 소유권을 클로저 안으로 옮긴다. 클로저가 동일한 변수에 대해 한 번만 소유권을 얻을 수 있어서 Once라는 이름이 들어갔다.

Fn은 캡처한 값을 불변으로 빌려온다.

FnMut은 값을 가변으로 빌려와서 환경을 변경할 수 있다.

In [14]:
{
    let cap_var = 300; 
    let add_one_v2: FnOnce(u32)->u32 = |x: u32| -> u32 { x + cap_var};
    
    println!(" {} ", add_one_v2(100));
    
};

Error: trait objects must include the `dyn` keyword

Error: mismatched types

Error: the size for values of type `dyn FnOnce(u32) -> u32` cannot be known at compilation time

### move 키워드로 캡처값의 소유권을 강제로 갖기

In [18]:
use std::any::type_name;

fn type_of<T>(_: T) -> &'static str {
    type_name::<T>()
}

In [22]:
fn main() {
    let x = vec![1, 2, 3];

    let equal_to_x = move |z| z == x;

    println!("{:?}", type_of(&equal_to_x));

    let y = vec![1, 2, 3];

    assert!(equal_to_x(y));
}

In [23]:
main()

"&ctx::main::{{closure}}"


()

### 함수의 매개변수로 처리할 때 

- 표준 라이브러리는 Fn, FnMut, FnOnce 트레잇을 제공하고 모든 클로저는 이 트레잇 중 하나를 구현한다.



- impl FnOnce 트레이트로 정의

In [14]:
fn f(g: impl FnOnce() -> String) {
    println!("{}", g());
}

let mut s = String::from("foo");
let t = String::from("bar");

f(|| {
    s += &t;
    s
});

foobar


## 1-3 클로저 축약형으로 작성하기 

- 클로저를 정의할 때 매개변수의 자료형이나 반환값 등을 생략할 수 있다.
- 또한 한줄만 본문이 작성할 경우는 중괄호도 생략이 가능하다

### 함수와 클러저 정의 알아보기

- 함수 정의를 클로저로 변경하기
- 클로저는 이름이 없어서 활용하려면 변수에 할당해서 사용한다. 

In [31]:
fn main() {
    fn  add_one_v1   (x: u32) -> u32 { x + 1 }   // 함수 정의 
    
    let add_one_v2 = |x: u32| -> u32 { x + 1 };  // 클로저 정의 
    let add_one_v3 = |x|             { x + 1 };  // 클로저 정의 : 타입 에노테이션 생략 
    let add_one_v4 = |x|               x + 1  ;  // 하나의 문장이라 표현식으로 지정 
    println!(" add_one_v1 {} ", add_one_v1(100));
    println!(" add_one_v2 {} ", add_one_v2(100));
    println!(" add_one_v3 {} ", add_one_v3(100));
    println!(" add_one_v4 {} ", add_one_v4(100));
}

In [32]:
main()

 add_one_v1 101 
 add_one_v2 101 
 add_one_v3 101 
 add_one_v4 101 


()

### 클로저의 매개변수에 자료형 지정하기
- 매개변수의 자료형이 불명확할 때는 반드시 지정해줘야 한다. 

In [35]:
fn main() {
    let c  = |x:u32| {x+x};

    println!(" {} ",c(10))
}

In [36]:
main()

 20 


()

## 1-4 구조체 내부에 클로저 정의하기

### 두 개의 클로저를 정의하고 하나를 다른 클로저에서 처리하기 

In [2]:
fn main() {
    let c  = |x:u32| {x+x};
    let d  = |y| c(y);

    println!(" {} ",d(10))
}

In [4]:
main();

 20 


### 클로저를 반환할 때는 캡처된 변수를 이동시키기

In [7]:
fn main() {
    
    let d  = |y| {move |x| x+y};

    println!(" {} ",d(10)(20))
}

In [8]:
main();

 30 


## 1-5 스마트 포인터로 클로저 반환하기 

In [13]:
let f: Box<dyn Fn()> = Box::new(|| {println!("foo")});

In [14]:
f()

foo


()

In [15]:
let fc: Box<dyn Fn(u32)> = Box::new(|x| {println!("foo {}",x)});

In [16]:
fc(100)

foo 100


()

In [17]:
let fc: Box<dyn Fn(u32,u32)> = Box::new(|x,y| {println!("foo {}",x+y)});

In [18]:
fc(100,200)

foo 300


()

In [28]:
pub fn make_quadratic(a: f64, b: f64, c: f64) -> impl Fn(f64) -> f64 {
    move |x| a*x*x + b*x + c
}

In [36]:
{ 
    let quad_fn: impl Fn(f64) -> f64 = make_quadratic(5.0, 4.0, 3.0);
           //error: `impl Trait` only allowed in function and inherent method return types
    println!("{}", type_of(&quad_fn));
}

Error: `impl Trait` only allowed in function and inherent method return types, not in variable binding

In [35]:
{ 
    let quad_fn = make_quadratic(5.0, 4.0, 3.0);
    
    println!("{}", type_of(&quad_fn));
}

&ctx::make_quadratic::{{closure}}


()

In [29]:
pub fn call_make_quadratic(x: f64) -> f64 {
    let quad_fn = make_quadratic(5.0, 4.0, 3.0);
    quad_fn(x) 
}

In [30]:
call_make_quadratic(10.0)

543.0

In [32]:
pub fn make_quadratic_box(a: f64, b: f64, c: f64) -> Box<dyn Fn(f64) -> f64> {
    Box::new(make_quadratic(a, b, c))
}

In [33]:
pub fn make_quadratic_no_capture() -> impl Fn(f64) -> f64 {
    |x| 42.0*x*x + 84.0*x + 0.0
}

In [34]:
pub fn make_quadratic_box_no_capture() -> Box<dyn Fn(f64) -> f64> {
    Box::new(make_quadratic_no_capture())
}

In [63]:
fn main() {
    let x = 100; 
    fn add() {}
    let v_char = vec!['A','c','e'];
    let v_num = vec![1,2];
    let clo :fn()->u32 = || {100 };
    let fun : fn() = add;
    let clo_mo : Box<dyn Fn() -> i32> = Box::new(move || { 100 + x});

    print_variable_type(&v_char);
    print_variable_type(&v_char[0]);
    print_variable_type(&v_num[0]);   
    print_variable_type(&clo);  
    print_variable_type(&fun); 
    print_variable_type(&clo_mo);
}

fn print_variable_type<K: ?Sized>(_: &K) {
    println!("{:?}", std::any::type_name::<K>())
}

In [64]:
main()

"alloc::vec::Vec<char>"
"char"
"i32"
"fn() -> u32"
"fn()"
"alloc::boxed::Box<dyn core::ops::function::Fn<()>+Output = i32>"


()